In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
from datasets import load_dataset

# Load the full corpus
dataset = load_dataset("mohres/The_Arabic_E-Book_Corpus", split="train")

# Print metadata for the first few books to choose one
for i in range(500):
    print(dataset[i]['title'], dataset[i]['author'], dataset[i]["booknr"])

README.md:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

data-00000-of-00002.arrow:   0%|          | 0.00/347M [00:00<?, ?B/s]

data-00001-of-00002.arrow:   0%|          | 0.00/346M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1745 [00:00<?, ? examples/s]

لؤلؤة الحب هربرت جورج ويلز 13028368
مغامرة العميل المرموق آرثر كونان دويل 13035305
ثلاثة دروس في ديكارت ألكسندر كواريه 13050381
كشف المُخبَّا عن فنون أوربا أحمد فارس الشدياق 13068491
حقائق الإسلام وأباطيل خصومه عباس محمود العقاد 13082050
مدينة زحلة عيسى إسكندر المعلوف 13149739
لاسلكيًّا روديارد كبلينج 13153615
مغامرة راكبة الدراجة في الطريق المهجور آرثر كونان دويل 13181417
الفاروق عمر محمد حسين هيكل 13538483
إبراهيم الثاني إبراهيم عبد القادر المازني 13572470
الطبيعة وما بعد الطبيعة: المادة . الحياة . الله يوسف كرم 13572791
الحجاج بن يوسف جُرجي زيدان 13595164
المسألة السودانية عمر طوسون 13603829
غَادَةُ كرْبَلاء جُرجي زيدان 13604281
أبيض الناب جاك لندن 13639135
أنت من فعلها! إدجار آلان بو 13649293
البدوية إبراهيم رمزي 13680727
قصة الأيام القادمة هربرت جورج ويلز 13736816
ألف ليلة وليلة ألف ليلة وليلة 13815802
نماذج بشرية أحمد رضا حوحو 13919315
تذكار الصبا: ذكرى ١٩ مارس محمد لطفي جمعة 13919490
الإسلام والحضارة الإنسانية عباس محمود العقاد 13920718
رواية الشقيقتين هنري لامنس 13925094
فَلسَف

In [3]:
x =dataset["booknr"]
i=0
for nr in dataset["booknr"]:
  if nr == 13815802:
    book = dataset[i]
    text = book['text']
    # Save the text to a file
    with open("selected_book.txt", "w", encoding="utf-8") as f:
        f.write(text)
  i +=1

In [4]:
import re

def split_into_paragraphs(input_file, output_file):
    # Read the text from the input file with UTF-8 encoding to support Arabic characters
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    # Split the text into sentences using a regex pattern
    # The pattern (?<=[.؟!])\s+ splits on whitespace that follows a period (.), Arabic question mark (؟), or exclamation point (!)
    # This keeps the punctuation with each sentence
    sentences = [s for s in re.split(r'(?<=[.؟!])\s+', text) if s.strip()]

    # Group sentences into paragraphs
    paragraphs = []
    for i in range(0, len(sentences), 4):
        # Take a slice of up to 4 sentences
        paragraph = sentences[i:i+3]
        # If this is not the first group and it has only 1 sentence, append it to the previous paragraph
        if i > 0 and len(paragraph) == 1:
            paragraphs[-1].append(paragraph[0])
        else:
            # Otherwise, add the new paragraph to the list
            paragraphs.append(paragraph)

    # Convert each paragraph (list of sentences) into a single string by joining sentences with spaces
    paragraph_texts = [' '.join(para) for para in paragraphs]

    # Write the paragraphs to the output file, each separated by a blank line
    with open(output_file, 'w', encoding='utf-8') as f:
        for para_text in paragraph_texts:
            f.write(para_text + '\n\n')

if __name__ == '__main__':
    # Define input and output filenames
    input_file = 'selected_book.txt'
    output_file = 'splitted_book.txt'
    # Call the function to process the file
    split_into_paragraphs(input_file, output_file)

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
import os
from tqdm import tqdm

def generate_embeddings(input_file, output_file, batch_size=32, model_name='sentence-transformers/LaBSE'):
    """
    Generate sentence embeddings for paragraphs in an input file using a multilingual model optimized for Arabic.

    Args:
        input_file (str): Path to the input file containing paragraphs.
        output_file (str): Path to save the embeddings as a NumPy array.
        batch_size (int): Number of paragraphs to process per batch (default: 32).
        model_name (str): Name of the SentenceTransformer model (default: 'sentence-transformers/LaBSE').

    Returns:
        None

    Raises:
        FileNotFoundError: If the input file does not exist.
        ValueError: If no valid paragraphs are found or embeddings are invalid.
    """
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file {input_file} not found.")

    print(f"Loading model: {model_name}")
    model = SentenceTransformer(model_name)
    print("Model loaded.")

    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    paragraphs = [para.strip() for para in content.split('\n\n') if para.strip()]
    if not paragraphs:
        raise ValueError("No valid paragraphs found in the input file.")
    print(f"File read, number of paragraphs: {len(paragraphs)}")

    embeddings = []
    for i in tqdm(range(0, len(paragraphs), batch_size), desc="Generating embeddings", total=(len(paragraphs) // batch_size + 1)):
        batch = paragraphs[i:i + batch_size]
        batch_embeddings = model.encode(batch, normalize_embeddings=True)
        embeddings.extend(batch_embeddings)

    embeddings = np.array(embeddings)
    if embeddings.shape[0] != len(paragraphs):
        raise ValueError(f"Expected {len(paragraphs)} embeddings, got {embeddings.shape[0]}.")
    if np.any(np.isnan(embeddings)):
        raise ValueError("Embeddings contain NaN values.")

    np.save(output_file, embeddings)
    print(f"Embeddings saved to {output_file}, shape: {embeddings.shape}")




input_file = '/content/splitted_book.txt'
output_file = 'embeddings.npy'
# Generate embeddings
generate_embeddings(input_file, output_file)

Loading model: sentence-transformers/LaBSE


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Model loaded.
File read, number of paragraphs: 467


Generating embeddings: 100%|██████████| 15/15 [00:03<00:00,  4.76it/s]

Embeddings saved to embeddings.npy, shape: (467, 768)


In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.1 MB/s eta 0:00:00


In [7]:
!pip list | grep faiss

faiss-cpu                             1.11.0


In [8]:
import faiss


import numpy as np

# Load the embeddings
embeddings = np.load('embeddings.npy').astype('float32')


# Get the dimensionality of the vectors (e.g., 512)
dimension = embeddings.shape[1]

# Create the FAISS index
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings)

# Save the index to a file
faiss.write_index(index, 'faiss_index.bin')

In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Load paragraphs
with open('splitted_book.txt', 'r', encoding='utf-8') as f:
    content = f.read()
paragraphs = [para.strip() for para in content.split('\n\n') if para.strip()]

# Load embeddings and FAISS index
embeddings = np.load('embeddings.npy').astype('float32')
index = faiss.read_index('faiss_index.bin')

# Load the SentenceTransformers model
model = SentenceTransformer('sentence-transformers/LaBSE')

# Set up TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(paragraphs)


In [10]:
def classical_search(query, top_k=5):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(paragraphs[i], similarities[i]) for i in top_indices]

def semantic_search(query, top_k=5):
    query_embedding = model.encode([query], convert_to_tensor=False).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    return [(paragraphs[i], distances[0][j]) for j, i in enumerate(indices[0])]

def display_results(classical_results, semantic_results):
    print("\nنتائج البحث")

    for i in range(5):
        # Classical search result
        c_text = classical_results[i][0]
        c_score = classical_results[i][1]
        # Semantic search result
        s_text = semantic_results[i][0]
        s_distance = semantic_results[i][1]

        print("-"*20)
        print(f"\nالنتائج {i + 1}:")
        print(f"\nClassical search result: {c_text}")
        print(f"Score: {c_score}")
        print(f"\nSemantic search result: {s_text}")
        print(f"Distance: {s_distance}")
        print("-"*20)


In [11]:
while True:
    query = input("\nأدخل استعلامك بالعربية (أو 'quit' للخروج): ")
    if query.lower() == 'quit':
        break
    if not query.strip():
        print("الرجاء إدخال استعلام صالح.")
        continue

    # Perform searches
    classical_results = classical_search(query)
    semantic_results = semantic_search(query)

    # Display results
    print(f"\nنتائج البحث للاستعلام: {query}")
    display_results(classical_results, semantic_results)


أدخل استعلامك بالعربية (أو 'quit' للخروج): من هم شهريار وشاهزمان

نتائج البحث للاستعلام: من هم شهريار وشاهزمان

نتائج البحث
--------------------

النتائج 1:

Classical search result: يُحكى أنه في قديم الزمان عاش ملكان عظيمان شقيقان يُدعيان شهريار وشاهزمان. كان شهريار هو الأكبر سنًّا، وامتد سلطانه إلى أقاصي الأرض. شعر شهريار برغبة في رؤية أخيه، فطلب من كبير وزرائه الذهاب لإحضاره إليه.
Score: 0.32989686491101883

Semantic search result: قصة الملك شهريار وشهرزاد
Distance: 1.081768274307251
--------------------
--------------------

النتائج 2:

Classical search result: قصة الملك شهريار وشهرزاد
Score: 0.27742265750562584

Semantic search result: يُحكى أنه في قديم الزمان عاش ملكان عظيمان شقيقان يُدعيان شهريار وشاهزمان. كان شهريار هو الأكبر سنًّا، وامتد سلطانه إلى أقاصي الأرض. شعر شهريار برغبة في رؤية أخيه، فطلب من كبير وزرائه الذهاب لإحضاره إليه.
Distance: 1.3581560850143433
--------------------
--------------------

النتائج 3:

Classical search result: مرت الأيام، وشاهزمان يزداد نحولًا، وا

In [12]:
import openai
from openai import OpenAI
from google.colab import userdata

# Load the API key from Colab secrets
client = OpenAI(api_key=userdata.get("openai-colab"))

def ask_arabic(question, context=None, model="gpt-4o-mini", temperature=0.0, max_tokens=128):
    if context:
        prompt = f"السياق: {context}\nالسؤال: {question}\nالإجابة:"
    else:
        prompt = f"السؤال: {question}\nالإجابة:"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "أجب على الأسئلة باللغة العربية."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=max_tokens
    )

    return response.choices[0].message.content.strip()


In [13]:
print(ask_arabic("ما هي عاصمة فرنسا"))

عاصمة فرنسا هي باريس.


In [14]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Load paragraphs
with open('splitted_book.txt', 'r', encoding='utf-8') as f:
    content = f.read()
paragraphs = [para.strip() for para in content.split('\n\n') if para.strip()]

# Load embeddings and FAISS index
embeddings = np.load('embeddings.npy').astype('float32')
index = faiss.read_index('faiss_index.bin')

# Load the SentenceTransformers model
model = SentenceTransformer('sentence-transformers/LaBSE')

# Set up TF‑IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(paragraphs)

def classical_search(query, top_k=3):
    qv = vectorizer.transform([query])
    sims = cosine_similarity(qv, tfidf_matrix).flatten()
    idxs = sims.argsort()[-top_k:][::-1]
    return list(zip(idxs, sims[idxs]))

def semantic_search(query, top_k=3):
    q_emb = model.encode([query], convert_to_tensor=False).astype('float32')
    dists, idxs = index.search(q_emb, top_k)
    return list(zip(idxs[0], dists[0]))

def get_window(idx):
    """Return the concatenation of paragraphs[idx-1], [idx], [idx+1], handling edges."""
    parts = []
    for j in (idx-1, idx, idx+1):
        if 0 <= j < len(paragraphs):
            parts.append(paragraphs[j])
    return "\n\n".join(parts)

# Interactive query loop
while True:
    query = input("\nأدخل استعلامك بالعربية (أو 'quit' للخروج): ")
    if query.lower() == 'quit':
        break
    if not query.strip():
        print("الرجاء إدخال استعلام صالح.")
        continue

    # 1) Baseline
    print("\n" + "="*80)
    print("الإجابة بدون سياق:")
    print(ask_arabic(query))
    print("-"*80)

    # 2) Classical with top‑3 windows
    classical_hits = classical_search(query, top_k=3)
    print("الإجابة مع سياقات أول ثلاث نتائج البحث الكلاسيكي (TF‑IDF):")
    for rank, (idx, score) in enumerate(classical_hits, 1):
        ctx = get_window(idx)
        ans = ask_arabic(query, ctx)
        print(f"\n#{rank} (score={score:.4f}):\n{ans}")
    print("-"*80)

    # 3) Semantic with top‑3 windows
    semantic_hits = semantic_search(query, top_k=3)
    print("الإجابة مع سياقات أول ثلاث نتائج البحث الدلالي (Embedding‑Based):")
    for rank, (idx, dist) in enumerate(semantic_hits, 1):
        ctx = get_window(idx)
        ans = ask_arabic(query, ctx)
        print(f"\n#{rank} (distance={dist:.4f}):\n{ans}")
    print("="*80)



أدخل استعلامك بالعربية (أو 'quit' للخروج): من هم شهريار وشاهزمان؟

الإجابة بدون سياق:
شهريار وشاهزمان هما شخصيتان رئيسيتان في مجموعة من القصص المعروفة باسم "ألف ليلة وليلة". 

- **شهريار**: هو ملك يكتشف خيانة زوجته، مما يدفعه إلى اتخاذ قرار بالزواج من امرأة جديدة كل ليلة ثم إعدامها في الصباح التالي. هذا السلوك يستمر حتى يتزوج من شهرزاد.

- **شاهزمان**: هو شقيق شهريار، ويظهر في بعض القصص كملك آخر. يشارك في الأحداث التي تتعلق بشهريار ويعكس بعض التوترات العائلية بينهما
--------------------------------------------------------------------------------
الإجابة مع سياقات أول ثلاث نتائج البحث الكلاسيكي (TF‑IDF):

#1 (score=0.3299):
شهريار وشاهزمان هما ملكان عظيمان شقيقان. شهريار هو الأكبر سنًا ويمتد سلطانه إلى أقاصي الأرض، بينما شاهزمان هو الأخ الأصغر الذي يعاني من الحزن بسبب هجر زوجته الملكة له.

#2 (score=0.2774):
شهريار وشاهزمان هما ملكان عظيمان شقيقان. شهريار هو الأكبر سنًا، ويمتد سلطانه إلى أقاصي الأرض، بينما شاهزمان هو الأخ الأصغر.

#3 (score=0.1785):
شهريار وشاهزمان هما ملكان عظيمان شقي